In [1]:
import pandas as pd

In [181]:
import matplotlib.pyplot as plt
%matplotlib inline

In [120]:
test_df = pd.read_csv('test_data.csv')

In [134]:
test_df

,Unnamed: 0,Date,y,temp,dew,humidity,cloudcover,windspeed,winddir,visibility
0,168354,2020-10-19 16:45:00,6699.14,32.0,10.0,25.82,0.0,14.8,300.0,4.0
1,168355,2020-10-19 17:00:00,6749.01,30.9,13.2,33.88,0.0,7.4,303.0,3.1
2,168356,2020-10-19 17:15:00,6409.50,30.9,13.2,33.88,0.0,7.4,303.0,3.1
3,168357,2020-10-19 17:30:00,6576.38,30.9,13.2,33.88,0.0,7.4,303.0,3.1
4,168358,2020-10-19 17:45:00,6282.79,30.9,13.2,33.88,0.0,7.4,303.0,3.1
...,...,...,...,...,...,...,...,...,...,...
42072,210426,2021-12-31 22:45:00,4712.55,10.0,8.0,87.38,27.4,0.0,0.0,2.0
42073,210427,2021-12-31 23:00:00,4649.58,9.2,7.3,88.04,28.5,0.0,0.0,1.3
42074,210428,2021-12-31 23:15:00,4571.39,9.2,7.3,88.04,28.5,0.0,0.0,1.3
42075,210429,2021-12-31 23:30:00,4510.39,9.2,7.3,88.04,28.5,0.0,0.0,1.3


In [125]:
test_df.rename(columns={'ds':'Date'}, inplace=True)

In [148]:
df_added = pd.DataFrame(test_df['Date'].append(pd.Series(pd.date_range('2022-01-01 00:00:00', '2022-01-31 23:45:00', freq='15min'))))

In [149]:
df_added.rename(columns={0: 'Date'}, inplace=True)

In [150]:
df_added = df_added.merge(test_df, left_on='Date', right_on = 'Date', how='left')

In [151]:
for i in range(42077, len(df_added['temp'])):
    df_added['temp'].iloc[i] = df_added['temp'].rolling(window=7, min_periods=1).mean().mean()
    df_added['dew'].iloc[i] = df_added['dew'].rolling(window=7, min_periods=1).mean().mean()
    df_added['humidity'].iloc[i] = df_added['humidity'].rolling(window=7, min_periods=1).mean().mean()
    df_added['cloudcover'].iloc[i] = df_added['cloudcover'].rolling(window=7, min_periods=1).mean().mean()
    df_added['windspeed'].iloc[i] = df_added['windspeed'].rolling(window=7, min_periods=1).mean().mean()
    df_added['winddir'].iloc[i] = df_added['winddir'].rolling(window=7, min_periods=1).mean().mean()
    df_added['visibility'].iloc[i] = df_added['visibility'].rolling(window=7, min_periods=1).mean().mean()

C:\Users\HP\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [154]:
df_added.drop(columns=['y', 'Unnamed: 0'], inplace=True)

In [155]:
df_added

,Date,temp,dew,humidity,cloudcover,windspeed,winddir,visibility
0,2020-10-19 16:45:00,32.000000,10.000000,25.820000,0.000000,14.800000,300.000000,4.000000
1,2020-10-19 17:00:00,30.900000,13.200000,33.880000,0.000000,7.400000,303.000000,3.100000
2,2020-10-19 17:15:00,30.900000,13.200000,33.880000,0.000000,7.400000,303.000000,3.100000
3,2020-10-19 17:30:00,30.900000,13.200000,33.880000,0.000000,7.400000,303.000000,3.100000
4,2020-10-19 17:45:00,30.900000,13.200000,33.880000,0.000000,7.400000,303.000000,3.100000
...,...,...,...,...,...,...,...,...
45048,2022-01-31 22:45:00,23.819772,15.781895,66.344152,32.177994,6.878087,152.905533,2.526586
45049,2022-01-31 23:00:00,23.819772,15.781895,66.344152,32.177994,6.878087,152.905533,2.526586
45050,2022-01-31 23:15:00,23.819772,15.781895,66.344152,32.177994,6.878087,152.905533,2.526586
45051,2022-01-31 23:30:00,23.819772,15.781895,66.344152,32.177994,6.878087,152.905533,2.526586


In [156]:
df_future = df_added.tail(31*96)

In [160]:
df_future = df_future.reset_index().drop(columns='index')

In [162]:
from prophet.serialize import model_to_json, model_from_json

with open('Partial_mv_model.json', 'r') as fin:
    saved_mv_model = model_from_json(fin.read())  # Load model

Importing plotly failed. Interactive plots will not work.


In [169]:
df_future['cap'] = 6000

In [170]:
df_future.rename(columns={'Date':'ds'}, inplace=True)

In [171]:
forecast = saved_mv_model.predict(df_future)

In [172]:
df_future.to_csv('Jan_2022_Data_MA_filled.csv')

In [173]:
forecast

,ds,trend,cap,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,...,windspeed,windspeed_lower,windspeed_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2022-01-01 00:00:00,5999.988099,6000,5349.897526,7595.555623,5999.987438,5999.988697,475.860089,475.860089,475.860089,...,10.582838,10.582838,10.582838,340.992023,340.992023,340.992023,0.0,0.0,0.0,6475.848188
1,2022-01-01 00:15:00,5999.988100,6000,5371.886380,7622.297658,5999.987439,5999.988698,460.808154,460.808154,460.808154,...,10.580604,10.580604,10.580604,340.905363,340.905363,340.905363,0.0,0.0,0.0,6460.796254
2,2022-01-01 00:30:00,5999.988101,6000,5362.903979,7500.734230,5999.987440,5999.988699,449.625721,449.625721,449.625721,...,10.579572,10.579572,10.579572,340.817959,340.817959,340.817959,0.0,0.0,0.0,6449.613822
3,2022-01-01 00:45:00,5999.988102,6000,5268.599337,7615.732320,5999.987441,5999.988699,442.060340,442.060340,442.060340,...,10.579041,10.579041,10.579041,340.729810,340.729810,340.729810,0.0,0.0,0.0,6442.048441
4,2022-01-01 01:00:00,5999.988102,6000,5412.462704,7525.626871,5999.987442,5999.988700,437.811731,437.811731,437.811731,...,10.578777,10.578777,10.578777,340.640918,340.640918,340.640918,0.0,0.0,0.0,6437.799833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2971,2022-01-31 22:45:00,5999.990341,6000,5079.850632,7362.821225,5999.989755,5999.990881,230.520340,230.520340,230.520340,...,10.578630,10.578630,10.578630,11.294087,11.294087,11.294087,0.0,0.0,0.0,6230.510681
2972,2022-01-31 23:00:00,5999.990341,6000,5038.004676,7321.339224,5999.989756,5999.990881,198.760415,198.760415,198.760415,...,10.578630,10.578630,10.578630,11.516449,11.516449,11.516449,0.0,0.0,0.0,6198.750756
2973,2022-01-31 23:15:00,5999.990342,6000,5054.699731,7316.123629,5999.989756,5999.990882,169.812271,169.812271,169.812271,...,10.578630,10.578630,10.578630,11.738406,11.738406,11.738406,0.0,0.0,0.0,6169.802613
2974,2022-01-31 23:30:00,5999.990343,6000,5037.932131,7329.417227,5999.989757,5999.990883,144.357354,144.357354,144.357354,...,10.578630,10.578630,10.578630,11.959958,11.959958,11.959958,0.0,0.0,0.0,6144.347696


In [174]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

,ds,yhat,yhat_lower,yhat_upper
0,2022-01-01 00:00:00,6475.848188,5349.897526,7595.555623
1,2022-01-01 00:15:00,6460.796254,5371.886380,7622.297658
2,2022-01-01 00:30:00,6449.613822,5362.903979,7500.734230
3,2022-01-01 00:45:00,6442.048441,5268.599337,7615.732320
4,2022-01-01 01:00:00,6437.799833,5412.462704,7525.626871
...,...,...,...,...
2971,2022-01-31 22:45:00,6230.510681,5079.850632,7362.821225
2972,2022-01-31 23:00:00,6198.750756,5038.004676,7321.339224
2973,2022-01-31 23:15:00,6169.802613,5054.699731,7316.123629
2974,2022-01-31 23:30:00,6144.347696,5037.932131,7329.417227
